# **Gradient Python SDK Tutorial**

This Notebook provides a step-by-step walkthrough of the Gradient Python SDK.  This library allows you to programmatically interact with Gradient from within a Jupyter Notebook environment (like this tutorial) or from within any python project. The Gradient Python SDK supplements the Gradient CLI functionality and UI with the added ability to automate actions and pipelines.

In this example, we'll be training a convolutional neural network to recognize handwritten digits using the canonical MNIST dataset and TensorFlow.  This demo walks through training a model, storing it in Gradient, deploying it as a RESTful API endpoint, and then making a prediction.  The purpose of the tutorial is to demonstrate how simple it is to build a machine learning pipeline from start to finish.  The code in this example lives in this GitHub repo: https://github.com/Paperspace/mnist-sample.git

_Note: This example takes approximately 10 mintues to run._

# Getting started
## Import the SDK Client

In [ ]:
# Import the SDK Client from the Gradient package (already installed in this Notebook)
# If using a new Notebook, install Gradient first with "pip install gradient"
from gradient import sdk_client

## Connect your account
You must create an [API key](https://docs.paperspace.com/gradient/get-started/install-the-cli#obtaining-an-api-key) in your account

In [ ]:
# Insert your API key between the quotes
api_key = ""

## Instantiate the SDK clients

In [ ]:
client = sdk_client.SdkClient(api_key)

# Or access them all from a single client
deployment_client = sdk_client.DeploymentsClient(api_key)
models_client = sdk_client.ModelsClient(api_key)
jobs_client = sdk_client.JobsClient(api_key)
projects_client = sdk_client.ProjectsClient(api_key)
experiment_client = sdk_client.ExperimentsClient(api_key)

## Create a Project

[Projects](https://docs.paperspace.com/gradient/projects/about) are a collection of your Experiments, Models, and Deployments. You can have an unlimited number of Projects within your account.  

In [ ]:
# Create a project. We'll call this project MNIST since we're working with the MNIST dataset
project_id = client.projects.create("MNIST")

# Training

There are two types of experiments in Gradient:
- [Singlenode](https://docs.paperspace.com/gradient/experiments/run-experiments-cli#creating-a-singlenode-experiment-using-the-cli): Train on a single instance
- [Multinode](https://docs.paperspace.com/gradient/experiments/multi-node-training): Train on multiple instances with distributed training

In this example, we recommend choosing one or the other.  After the model is done training, the subsequent steps of capturing the model and deploying it as an API endpoint will be the same.

## Run a basic singlenode experiment
### (See below for a distributed training example)

In [ ]:
# Create a dictionary of parameters for running an experiment
env = {
    "EPOCHS_EVAL":5,
    "TRAIN_EPOCHS":10,
    "MAX_STEPS":1000,
    "EVAL_SECS":10
}

parameters = { 
    "name": "mnist",
    "project_id": (project_id),
    "container": "tensorflow/tensorflow:1.13.1-gpu-py3",
    "machine_type": "P4000",
    "command": "pip install -r requirements.txt && python mnist.py",
    "workspace_url": "https://github.com/Paperspace/mnist-sample.git", #note: you can specify a git repo or a specific git commit, a local directory, or even an S3 bucket
    "model_path": "/storage/models/tutorial-mnist/",
    "model_type": "Tensorflow"
}

# Pass dictionary into experiments client
experiment_id = client.experiments.run_single_node(**parameters)

# Display the experiment details
print(client.experiments.get(experiment_id))

# Output the path so you can open the experiment in the UI
print("https://www.paperspace.com/console/" + project_id + "/experiments/" + experiment_id)

## Scale up with distributed training
##### Notice how few changes are required to go from a _singlenode experiment_ to a _multinode experiment_

In [ ]:
# Create a dictionary of parameters for running an experiment
env = {
    "EPOCHS_EVAL":5,
    "TRAIN_EPOCHS":10,
    "MAX_STEPS":1000,
    "EVAL_SECS":10
}

multi_node_parameters = { 
    "name": "multinode_mnist",
    "project_id": (project_id),
    "experiment_type_id": 2,
    "worker_container": "tensorflow/tensorflow:1.13.1-gpu-py3",
    "worker_machine_type": "K80",
    "worker_command": "pip install -r requirements.txt && python mnist.py",
    "experiment_env": env,
    "worker_count": 2,
    "parameter_server_container": "tensorflow/tensorflow:1.13.1-gpu-py3",
    "parameter_server_machine_type": "K80",
    "parameter_server_command": "pip install -r requirements.txt && python mnist.py",
    "parameter_server_count": 1,
    "workspace_url": "https://github.com/Paperspace/mnist-sample.git", #note: you can specify a git repo or a specific git commit, a local directory, or even an S3 bucket
    "model_path": "/storage/models/tutorial-mnist/",
    "model_type": "Tensorflow"
}

# Pass dictionary into experiments client
experiment_id = client.experiments.run_multi_node(**multi_node_parameters)
client.experiments.get(experiment_id)

# Display the experiment details
print(client.experiments.get(experiment_id))

# Output the path so you can open the experiment in the UI
print("https://www.paperspace.com/console/" + project_id + "/experiments/" + experiment_id)

### Optional: Watch the state of the Experiment 
#### You can also navigate to the UI to see the state

In [ ]:
from gradient import constants
# Stream the state of the experiment
print("Watching state of experiment")
state = ""
while state != "running":
    new_state = constants.ExperimentState.get_state_str(client.experiments.get(experiment_id).state)
    if new_state != state:
        print("state: "+new_state)
        state = new_state

### Optional: Stream the logs within Jupyter
#### You can also navigate to the UI to view the logs

In [ ]:
log_streamer = client.experiments.yield_logs(experiment_id)
# Create a log stream & print all logs for the duration of experiment
print("Streaming logs of experiment")
try:
    while True:
        print(log_streamer.send(None))
except:
    print("done streaming logs")

## Inspect your model

In [ ]:
model = client.models.list(experiment_id = experiment_id)
print(model)

#### View the accuracy

In [ ]:
model[0].summary['accuracy']['result']

# Serve your model as an API endpoint
### Create Deployment

In [ ]:
deploy_param = {
    "deployment_type" : "Tensorflow Serving on K8s",
    "image_url": "tensorflow/serving:latest-gpu",
    "name": "sdk_tutorial",
    "machine_type": "K80",
    "instance_count": 2,
    "model_id" : model[0].id
}
mnist = client.deployments.create(**deploy_param)

### Start the Deployment

In [ ]:
client.deployments.start(mnist)

### List your Deployments

In [ ]:
deployment = client.deployments.list(model_id=model[0].id)
deployment

### Get the endpoint

In [ ]:
print(deployment)
print("Endpoint: "+deployment[0].endpoint)

# Make a prediction
#### Run the prerequisite inference code

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image as pilimage
import requests

def get_image_from_drive(path):
    # Load the image
    image = pilimage.open(path)
    image = image.convert('L')
    image = np.resize(image, (28,28,1))
    image = np.array(image)
    image = image.reshape(28,28)
    return image

def show_selected_image(image):
    fig = plt.figure()
    plt.subplot(1, 1, 1)
    plt.tight_layout()
    plt.imshow(image, cmap='gray', interpolation='none')
    plt.xticks([])
    plt.yticks([])
    plt.show()


def make_vector(image):
    vector = []
    for item in image.tolist():
        vector.extend(item)
    return vector


def make_prediction_request(image, prediction_url):
    vector = make_vector(image)
    json = {
        "inputs": [vector]
    }
    response = requests.post(prediction_url, json=json)

    print('HTTP Response %s' % response.status_code)
    print(response.text)

#### Load and display the image to validate the results

In [ ]:
image = get_image_from_drive('mnist_5.png')
show_selected_image(image)

### Make the prediction!

In [ ]:
make_prediction_request(image, deployment[0].endpoint)

#### One more time 😀  

In [ ]:
image = get_image_from_drive('mnist_3.png')
show_selected_image(image)
make_prediction_request(image, deployment[0].endpoint)